Muslimov Arthur, Last Checkpoint: 03/31/2020    <br/>
                 Last Checkpoint: 12/19/2021

Реальные данные редко бывают очищенными и однородными. В частности,       <br/>
во многих интересных наборах данные некоторое количество данных           <br/>
отсутствует. Ещё более затрудняет работу то, что в различных источниках   <br/>
отсутствующие данные могут быть помечены различным образом.        <br/>
                                                                          <br/>
В это разделе мы обсудим общие соображения, касающиеся отсутствующих      <br/>
данных, обсудим способы представления их библиотекой Pandas и             <br/>
продемонстрируем  встроенные инструменты библиотеки Pandas для обработки  <br/>
отсутствующих данных в языке Python. Здесь и далее мы будем называть      <br/>
отсутствующие данные ***null***, NaN или NA-значениями.

In [1]:
import numpy as np
import pandas as pd

%xmode Minimal
%autosave 0

Exception reporting mode: Minimal


Autosave disabled


# Компромиссы при обозначении отсутствующих данных

Существуют несколько схем для обозначения пропущенных данных.         <br/>
Они основаны на одной из двух стратегий: использование ***маски***,   <br/>
отмечающей отсутвующие данные, или ***значения-индикаторы***          <br/>
(sentinel value), говорящие, что здесь нет ничего важного.

Маска может быть или отдельным булевым массивом или отдельным  <br/>
битом у каждого значения, можно сказать, биркой данных.

Значением-индикатором может быть или специально отведённое значение,      <br/>
например 9999, или какай-нибудь редкая комбинация битов. Ещё им может     <br/>
стать более глобальное значение, например для `float` данных это `NaN` -  <br/>
\- специального значение, всключённого в спецификации IEEE для `float`.

В каждом из подходов есть свои компромиссы: маска требует дополнительной  <br/>
памяти и процессорного времени, а значение-индекатор сокращает диапозон   <br/>
доступных данных и может потребовать выполнения дополнительной            <br/>
(зачастую неоптимизированной) логики. Общие  специальные значения, такие  <br/>
как NaN, доступны далеко не для всех типов данных.

Как и в большинестве случав, где нет универсального решения, различные языки      <br/>
и системы использует различные методы. Например, язык R применяет заранее         <br/>
зарезервированные комбинации битов для каждого типа данных как бирка для          <br/>
отсутствующих данных. А система SciDB вообще к каждой ячейки данных присоединяет  <br/>
дополнительный байт для индикации NA состояния.

# Отсутствующие данные в библиотеки Pandas

Как мы знаем, Pandas основан на NumPy, а значит и зависит от него. А в NumPy   <br/>
есть только один встроенный индикатор - `NaN`, и он создан для `float` значений.

Pandas мог бы использовать метод языка R, задавая в качестве индикатора         <br/>
комбинации битов для каждого типа данных. Но, это было бы очень громоздким      <br/>
решением. Если в R всего четыре типа данных, то в NumPy их ***четырнадцать***!  <br/>
Это привело бы к громадным накладным расходам в разнообразных частных случаях   <br/>
операций, а может даже пришлось бы строить свою отдельную ветвь пакета NumPy.   <br/>
А выделение отдельного бита на маску будет критично для малобитных типов.

NumPy поддерживает использование маскированных массив, т.е. массивов данных,  <br/>
которым в придачу идут ещё и булевые массивы-маски, обозначающие "хорошие"    <br/>
и "плохие" данные. Но накладные расходы на хранение, вычисление и поддержку   <br/>
кода сделали этот вариант малопривлекательным.

В итоге в Pandas было решено использовать индикаторы,     <br/>
а также уже существующие пустые значения: `NaN` значение  <br/>
для `float` данных и объект `None` из Python. Здесь тоже  <br/>
есть свои недостатки, но на практике в большинстве        <br/>
случавет оно представляет собой удачный компромисс.

### None: отсутствующие данные в языке Python

Первое из используемых Pandas значений-индикаторов - None, объект-одиночка  <br/>
Python. Т.к. None - объект Python, его нельзя использовать в произвольных   <br/>
массивах NumPy/Pandas, а только в массивах с типом `object` из Python.

In [2]:
vals1 = np.array([1, None, 3, 4])
print('size in bytes:', vals1.nbytes)  # наверное отъест он много
vals1

size in bytes: 32


array([1, None, 3, 4], dtype=object)

Наилучшим общим типом NumPy посчитал объект Python. Это может быть     <br/>
полезно, т.к. теперь наш массив может хранить почти всё, но вся        <br/>
прелесть массивов улетучилась, т.к. теперь за логику отвечает Python.

In [3]:
print("object:")
%timeit np.arange(1e6, dtype=object).sum()
                    # или 1000000 (1m), а можно ещё 1E6
%timeit np.arange(1e6, dtype=int).sum()

object:
55.7 ms ± 281 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
2.27 ms ± 112 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


Ещё `None` не знает как вести себя при операциях с числами, поэтому будет исключение.

In [4]:
try:  # давненько я этим не пользовался
    vals1 + 1
except TypeError:
    print("End of the world!")

End of the world!


### NaN: отсутствующие числовые данные

Ещё одно уже существующее пустое значение - это `NaN`. Это специальное значение  <br/>
с плавающей точкой, распознаваемое всеми системами, испольующими стандартное     <br/>
IEEE-представление чисел с плавающей точкой (в общем, IEEE - это религия).

In [7]:
vals2 = np.array([1, np.nan, 3, 4])   # здесь уже будет строгая типизация, а значит и хорошая
                                      #   скорость
print("size in bytes:", vals2.nbytes) # потреблять, конечно, здесь он станет не меньше, т.к. по
print("type:", vals2.dtype)           #   умолчанию NumPy поставит тип float64, но можно и
                                      #   ограничиться float16

size in bytes: 32
type: float64


Тип у нас установлен, и это не `object`. Это значит, операции будут проходить  <br/>
по скомпилированному коду. Но следует помнить, что `NaN` чем-то подобен        <br/>
"вирусу данных". Любой объект, к которому он "прикасается", становится `NaN`.

In [8]:
1 + np.nan  # Спасайся кто может!

nan

In [9]:
0 * np.nan  # Неубеваемая болезнь

nan

In [10]:
try:
    np.nan / 0  # Приходится идти на крайние меры
except ZeroDivisionError:
    print("Победа! Но какой ценой...")

Победа! Но какой ценой...


Это значит, что обычные операции агрегирования становятся бесполезными.

In [11]:
vals2.sum(), vals2.min(), vals2.max()  # Падшие города

(nan, nan, nan)

Но, как ты помнишь из таблицы прошлой главы, есть специализированные  <br/>
версии функций, игнорирующие пропущенные значения.

In [12]:
np.nansum(vals2), np.nanmin(vals2), np.nanmax(vals2)  # Вакцина найдена!

(8.0, 1.0, 4.0)

Не забывай, что `NaN` - значение с плавающей точкой.  <br/>
Аналога для других типов нет.

### Значения `NaN` и `None` в библиотеке Pandas

Как и у `NaN`, так и у `None` есть своё назначение,     <br/>
и Pandas делает их практически взаимозаменяемыми путём  <br/>
преобразования одного в другое в определённых случаях.

In [13]:
pd.Series([1, np.nan, 2, None])

0    1.0
1    NaN
2    2.0
3    NaN
dtype: float64

Лучше представлять так: если в массиве кроме `None` больше нет    <br/>
`object` объектов, то Pandas заменяет `None` на `np.nan` и всё    <br/>
сводится к `float`, т.к. он станет лучшим общим типом. Если же    <br/>
другие `object`-объекты всё-таки есть, то всё остаётся как есть,    <br/>
и оптимальным типов станет `object`.

In [14]:
x = pd.Series(range(2), dtype=int)
x  # чистокровный int-массив

0    0
1    1
dtype: int64

In [15]:
x[0] = None  # замещаем один int на object
x  # Pandas увидит это и сделает всё с наименьшими потерями

0    NaN
1    1.0
dtype: float64

Автор говорит, что в Pandas хотели добавить целочисленный NA,   <br/>
но на момент написания книги (т.е. 2016 год), ещё не включили.

> Хотя подобный подход со значениями-индикаторами/приведением типов   <br/>
> библиотеки Pandas может показаться несколько вычурным по сравнению  <br/>
> с более унифицированным подходом к NA-значениям в таких             <br/> 
> предметно-ориентированных языках, как R, на практике он прекрасно   <br/>
> работает и на моей памяти лишь изредка вызывал проблемы.

А вот и правила повышающего приведения типов в Pandas в случае наличия NA-значений.

**Класс типов**    | **Преобразование при хранениии NA-значений** | **Значение-индикатор NA**
:------------------|:---------------------------------------------|:-------------------------
С плавающей точкой | Без изменений                                | `np.nan`
Объект (object)    | Без изменений                                | `None` или `np.nan`
Целое число        | Приводится к `float64`                       | `np.nan`
Булево значение    | Приводится к `object`                        | `None` или `np.nan`

Имей ввиду, что строки и даже символы (которые по факту в Python тоже строки) - это `object`.

# Операции над пустыми значениями

Библиотека Pandas рассматривает значения `None` и `NaN` как     <br/>
взаимозаменямые пустые значения. Существует несколько методов,  <br/>
упрощающие жизнь при работе с массивами, хранящими их.

- `isnull()` - генерирует булеву маску для отсутствующих значений.
- `notnull()` - противоположность метода `isnull()`.
- `dropna()` - возвращает отфильтрованный вариант данных.
- `fillna()` - возвращает копию данных, в которой пропущенные значения заполнены или восстановлены.

Ну и на последок, кратко рассмотрим и продемонстрируем эти методы.

### Выявление пустых значений

У структур данных Pandas есть два удобным метода  <br/>
для получения маски "хороших" и "плохих" данных.

In [17]:
data = pd.Series([1, np.nan, 'hello', None])
data.isnull()  # Ищем угрозу

0    False
1     True
2    False
3     True
dtype: bool

Булевы маски садятся на Series и DataFrame  <br/>
также хорошо, как и на массивы NumPy.

In [18]:
data[data.notnull()]  # Ещё не испорченные объекты

0        1
2    hello
dtype: object

Использование этих методов такое же лёгкое и с DataFrame.

*Замечено мною позже:*

*Маски не ложаться на DataFrame также хорошо, если объекты Series в нём  <br/>
имеют разные типы. Да, DataFrame не имеем общего для себя типа.*         <br/>

*`NaN` - это 'numpy.float64'    <br/>
`np.nan` - это 'float'.*        <br/>
                                <br/>
*`In[68]: NaN == np.nan`        <br/>
`Out[68]: False`*

### Удаление пустых значений

Кроме маскирования, существуют ещё и методы для устранения NA-значений:  <br/>
`dropna()` для отбрасывания "плохих" значений и `fillna()` для их        <br/>
замены. С Series объектами всё очень даже просто.

In [19]:
data.dropna()  # остаются только хорошие данные

0        1
2    hello
dtype: object

С DataFrame всё чуть сложнее. По одному отбрасывать нельзя.  <br/>
По умолчанию удаляются строки с NA-значениями. Даже если оно одно.

In [20]:
df = pd.DataFrame([[1,      np.nan, 2],
                   [2,      3,      5],
                   [np.nan, 4,      6]])  # всё по красоте
df.dropna()  # удаляются заражённые строки. Даже если не все значения плохие

,0,1,2
1,2.0,3.0,5


Как альтернатива, можно удалять не строки, а столбцы, настроив `axis`.

In [22]:
df.dropna(axis='columns')  # или axis=1

,2
0,2
1,5
2,6


Если ты хочешь, чтобы  отбрасывались только строки и столбцы, вообще    <br/>
не имеющие "хороших" значений, или где "плохих" значений большинство,   <br/>
можешь тонко настроить функцию, задав аргументы `how` и `thresh`.

In [23]:
df[3] = np.nan  # добавляем столбик, содержащий только NA-значения
df

,0,1,2,3
0,1.0,NaN,2,NaN
1,2.0,3.0,5,NaN
2,NaN,4.0,6,NaN


In [25]:
df.dropna(axis=1, how='all')  # по умолчанию how=`any`

,0,1,2
0,1.0,NaN,2
1,2.0,3.0,5
2,NaN,4.0,6


И мы видим, что безнадёжный столбец не попал в результат. Аргумент    <br/>
`thresh` же даёт возможность выбрать минимум хороших значений         <br/>
для каждой строки или столбца, чтобы тот попал в выходной результат.

In [26]:
df.dropna(axis='rows', how='any', thresh=3)  # проходят только стоки, хранящие от 3 не NA-значения.

,0,1,2,3
1,2.0,3.0,5,NaN


Здесь отбросились первая и последняя строка, т.к. они  <br/>
не набрали нужное количество существенных значений.

### Заполнение пустых значений

Иногда всё-же предпочтительнее просто заменять NA-значения на что-то другое.        <br/>
Это можно сделать и с помощью метода `isnull()`, но это настолько распространённая  <br/>
операция, что её решили добавить нативно в Pandas как метод `fillna()`.             <br/>
Он возвращает копию данных с заменёнными пустыми значениями.

In [27]:
data = pd.Series([1, np.nan, 2, np.nan, 3], index=list('abcde'))
data

a    1.0
b    NaN
c    2.0
d    NaN
e    3.0
dtype: float64

In [28]:
data.fillna(0)  # заменяем все NA-значения на простой ноль, но оригинал не трогает

a    1.0
b    0.0
c    2.0
d    0.0
e    3.0
dtype: float64

Можно задать метод заполнения по направлению "вперёд", копируя  <br/>
предыдущее значение в ячейку с пустым значением.

In [29]:
data.fillna(method='ffill')  # расшифровывается как forward fill (~ передовое заполение).
                             # Наверное у создателей метода точка зрения на направление другая

a    1.0
b    1.0
c    2.0
d    2.0
e    3.0
dtype: float64

Также можно задать параметр по направлению "назад", копируя следующую ячейку.

In [30]:
data.fillna(method='bfill')  # расшифровывается как back fill

a    1.0
b    2.0
c    2.0
d    3.0
e    3.0
dtype: float64

С DataFrame метод работает также, но ещё и есть возможность выбрать ось.

In [31]:
df

,0,1,2,3
0,1.0,NaN,2,NaN
1,2.0,3.0,5,NaN
2,NaN,4.0,6,NaN


In [32]:
df.fillna(method='ffill', axis='columns')  # или axis=1

,0,1,2,3
0,1.0,1.0,2.0,2.0
1,2.0,3.0,5.0,5.0
2,NaN,4.0,6.0,6.0


Обрати внимание, что `NaN` слева не был замён. Это потому что  <br/>
ему неоткуда было взять значение, т.к. столбца перед ним нет.